In [ ]:
# wavファイルの再生 

import pyaudio
import wave

# streamで一度に読み込むデータ量のバッファ
CHUNK = 1024

# ファイルの読み込み
filename = "440Hz_standard.wav"
wf = wave.open(filename, 'rb')

# 初期化
p = pyaudio.PyAudio()
stream = p.open(
    format=p.get_format_from_width(wf.getsampwidth()), # 量子化ビット
    channels=wf.getnchannels(),                        # チャンネル数
    rate=wf.getframerate(),                            # サンプリング周波数
    output=True)                                       # 外部出力するか否か

# CHUNK分のデータだけ読み込む
data = wf.readframes(CHUNK)

# 音声の再生
while len(data) > 0:            # wavファイルのデータがなくなるまで読み込む
    stream.write(data)          # スピーカーに読み込んだデータを出力
    data = wf.readframes(CHUNK) # 次のデータを読み込む
    
# 終了処理
stream.stop_stream()
stream.close()
p.terminate()

In [ ]:
# リアルタイムでマイクの音の可視化

import pyaudio
# jupyterでリアルタイム描画する場合に必要
from IPython.display import display, clear_output

# 定数
CHUNK = 1024     # バッファ
sr = 2048        # サンプリング周波数
frame_count = 10 # 保持するフレーム数

# 横軸として経過時間を定義
t = np.linspace(CHUNK / sr * frame_count, 0, sr * frame_count)

# 初期化
## pyaudio
p = pyaudio.PyAudio()
stream = p.open(format = pyaudio.paInt16,  # 量子化ビット
                channels = 1,              # チャンネル数
                rate = sr,                 # サンプリング周波数
                frames_per_buffer = CHUNK, # 1フレームのデータ数
                input = True)              # 入力か否か

## 5秒間のデータを保持するためのバッファ
data = np.zeros(sr * frame_count)

## 描画領域の設定
plt.ylim(-1,1)
    
## 時系列に流れるグラフ
while stream.is_active():
    input = stream.read(CHUNK)                   # マイクへの入力データを読み込む
    input = np.frombuffer(input, dtype=np.int16) # 処理速度高速化するためndarrayに変換
    input = input / np.max(np.abs(input))        # 振幅が-1 ~ 1になるように正規化
    
    #  最新のフレームのみ保持する
    data = np.append(data, input)
    if len(data)/1024 > frame_count:
        data = data[1024:]
        
    # 描画
    plt.plot(t, data)
    display()               # jupyterの場合必要
    plt.pause(0.001)
    clear_output(wait=True) # jupyterの場合必要